# Configuration

## Import Libraries

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
import warnings
import time

# Suppress unnecessary warnings
warnings.filterwarnings("ignore", category=UserWarning)

### Import Torch2PC Library

In [ ]:
try:
    from Torch2PC import TorchSeq2PC as T2PC
except ImportError:
    print("Cloning Torch2PC repository...")
    import subprocess

    subprocess.run(
        ["git", "clone", "-q", "https://github.com/RobertRosenbaum/Torch2PC.git"]
    )
    from Torch2PC import TorchSeq2PC as T2PC
print("Torch2PC imported successfully.")

In [12]:
import sys

sys.path.append("../src")
from train import T2PC

## Set random seeds

In [13]:
torch.manual_seed(42)
np.random.seed(42)

## GPU Setup

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Code will run on: {device}")

## Data Loader

In [14]:
def load_mnist_data():
    """Load and preprocess the MNIST dataset."""
    # Define pre process
    mnist_transform = transforms.Compose(
        [
            transforms.Resize((32, 32)),  # resize from 28x28
            transforms.ToTensor(),  #  Converts the image to PyTorch tensors
            transforms.Normalize(
                (0.1307,), (0.3081,)
            ),  # Normalize pixel values using the M and DE of the MNIST dataset. This helps stabilize training
        ]
    )

    # Download the data (train and test)
    train_val_dataset = torchvision.datasets.MNIST(
        root="./data", train=True, transform=mnist_transform, download=True
    )
    test_dataset = torchvision.datasets.MNIST(
        root="./data", train=False, transform=mnist_transform, download=True
    )
    # Split train data into train (90%) and validation (10%)
    train_size = int(0.9 * len(train_val_dataset))
    val_size = len(train_val_dataset) - train_size
    train_dataset, val_dataset = torch.utils.data.random_split(
        train_val_dataset, [train_size, val_size]
    )
    # Define DataLoader instances with proper BATCH_SIZE and shuffle=True to prevent the model from learning the order of the data
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
    # Stablish the class names (0 to 9)
    class_names = [str(i) for i in range(10)]
    return train_loader, val_loader, test_loader, class_names

# Define Model

## Hyperparameters

In [15]:
# General
LATENT_DIM = 84
NUM_CLASSES = 10
BATCH_SIZE = 32
LEARNING_RATE = 0.001
NUM_EPOCHS = 15

# Predictive Coding Specific Hyperparameters
N_INFERENCE_STEPS = 5
INFERENCE_LEARNING_RATE = 5e-5

## Model Architecture

In [16]:
def create_model():
    """Creates a fresh instance of the CNN architecture."""
    return nn.Sequential(
        nn.Sequential(nn.Conv2d(1, 6, 5), nn.ReLU(), nn.MaxPool2d(2, 2)),
        nn.Sequential(nn.Conv2d(6, 16, 5), nn.ReLU(), nn.MaxPool2d(2)),
        nn.Sequential(nn.Flatten(), nn.Linear(16 * 5 * 5, 120), nn.ReLU()),
        nn.Sequential(nn.Linear(120, LATENT_DIM), nn.ReLU()),
        nn.Sequential(nn.Linear(LATENT_DIM, NUM_CLASSES)),
    )

# Training Functions

## Backpropagation

In [ ]:
def train_model_backprop(model, train_loader, val_loader, num_epochs=NUM_EPOCHS):
    """Train the CNN model using standard backpropagation."""
    history = {
        "train_losses": [],
        "train_accuracies": [],
        "val_losses": [],
        "val_accuracies": [],
        "epoch_times": [],
        "total_time": 0.0,
    }
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

    print(f"\nStarting standard backpropagation training...")
    print("-" * 60)

    total_start_time = time.time()

    for epoch in range(num_epochs):
        epoch_start_time = time.time()
        model.train()
        running_loss, correct_predictions, total_samples = 0.0, 0, 0
        for data, targets in train_loader:
            data, targets = data.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(data)  # Forward Pass
            loss = criterion(outputs, targets)
            loss.backward()  # Backward Pass
            optimizer.step()
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total_samples += targets.size(0)
            correct_predictions += (predicted == targets).sum().item()

        history["train_losses"].append(running_loss / len(train_loader))
        history["train_accuracies"].append(100.0 * correct_predictions / total_samples)

        model.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0
        with torch.no_grad():
            for data, targets in val_loader:
                data, targets = data.to(device), targets.to(device)
                outputs = model(data)
                loss = criterion(outputs, targets)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += targets.size(0)
                val_correct += (predicted == targets).sum().item()

        history["val_losses"].append(val_loss / len(val_loader))
        history["val_accuracies"].append(100.0 * val_correct / val_total)

        epoch_end_time = time.time()
        epoch_duration = epoch_end_time - epoch_start_time
        history["epoch_times"].append(epoch_duration)

        print(
            (
                f"Epoch [{epoch+1:02d}/{num_epochs}] -> "
                f"Train Loss: {history['train_losses'][-1]:.4f}, "
                f"Train Acc: {history['train_accuracies'][-1]:.2f}% | "
                f"Val Loss: {history['val_losses'][-1]:.4f}, "
                f"Val Acc: {history['val_accuracies'][-1]:.2f}% | "
                f"Time: {epoch_duration:.2f}s"
            )
        )

    total_end_time = time.time()
    history["total_time"] = total_end_time - total_start_time

    print("\nBackpropagation training completed!")
    print(f"Total training time: {history['total_time']:.2f} seconds ⏱️")

    return history

# Predictive Coding

In [17]:
def train_model_pc(model, train_loader, val_loader, num_epochs=NUM_EPOCHS):
    """Train the CNN model using Predictive Coding (Strict)."""
    history = {
        "train_losses": [],
        "train_accuracies": [],
        "val_losses": [],
        "val_accuracies": [],
        "epoch_times": [],
        "total_time": 0.0,
    }
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

    print(f"\nStarting Predictive Coding training...")
    print(
        f"Using eta={INFERENCE_LEARNING_RATE} and n={N_INFERENCE_STEPS} inference iterations."
    )
    print("-" * 60)

    total_start_time = time.time()

    for epoch in range(num_epochs):
        epoch_start_time = time.time()
        model.train()
        running_loss, correct_predictions, total_samples = 0.0, 0, 0
        for data, targets in train_loader:
            data, targets = data.to(device), targets.to(device)
            targets_onehot = F.one_hot(targets, num_classes=NUM_CLASSES).float()

            vhat, loss, _, _, _ = T2PC.PCInfer(
                model,
                criterion,
                data,
                targets_onehot,
                "Strict",
                eta=INFERENCE_LEARNING_RATE,
                n=N_INFERENCE_STEPS,
            )
            optimizer.step()
            optimizer.zero_grad()
            running_loss += loss.item()
            _, predicted_labels = torch.max(vhat[-1].data, 1)
            total_samples += targets.size(0)
            correct_predictions += (predicted_labels == targets).sum().item()

        history["train_losses"].append(running_loss / len(train_loader))
        history["train_accuracies"].append(100.0 * correct_predictions / total_samples)

        model.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0
        with torch.no_grad():
            for data, targets in val_loader:
                data, targets = data.to(device), targets.to(device)

                outputs = model(data)
                targets_onehot = F.one_hot(targets, num_classes=NUM_CLASSES).float()

                loss = criterion(outputs, targets_onehot)
                val_loss += loss.item()

                _, predicted = torch.max(outputs.data, 1)
                val_total += targets.size(0)
                val_correct += (predicted == targets).sum().item()

        history["val_losses"].append(val_loss / len(val_loader))
        history["val_accuracies"].append(100.0 * val_correct / val_total)

        epoch_end_time = time.time()
        epoch_duration = epoch_end_time - epoch_start_time
        history["epoch_times"].append(epoch_duration)

        print(
            (
                f"Epoch [{epoch+1:02d}/{num_epochs}] -> "
                f"Train Loss: {history['train_losses'][-1]:.4f}, "
                f"Train Acc: {history['train_accuracies'][-1]:.2f}% | "
                f"Val Loss: {history['val_losses'][-1]:.4f}, "
                f"Val Acc: {history['val_accuracies'][-1]:.2f}% | "
                f"Time: {epoch_duration:.2f}s"
            )
        )

    total_end_time = time.time()
    history["total_time"] = total_end_time - total_start_time

    print("\nPredictive Coding training completed!")
    print(f"Total training time: {history['total_time']:.2f} seconds ⏱️")

    return history

# Execution

In [18]:
# Load Data
train_loader, val_loader, test_loader, class_names = load_mnist_data()

# Backpropagation Run
# bp_model = create_model().to(device)
# bp_history = train_model_backprop(bp_model, train_loader, val_loader)

# Predictive Coding Run
pc_model = create_model().to(device)
pc_history = train_model_pc(pc_model, train_loader, val_loader)


Starting Predictive Coding training...
Using eta=5e-05 and n=5 inference iterations.
------------------------------------------------------------
Epoch [01/15] -> Train Loss: 0.0400, Train Acc: 81.49% | Val Loss: 0.0276, Val Acc: 88.50% | Time: 40.74s
Epoch [02/15] -> Train Loss: 0.0243, Train Acc: 89.51% | Val Loss: 0.0225, Val Acc: 90.35% | Time: 42.73s
Epoch [03/15] -> Train Loss: 0.0208, Train Acc: 90.95% | Val Loss: 0.0204, Val Acc: 91.15% | Time: 36.47s
Epoch [04/15] -> Train Loss: 0.0193, Train Acc: 91.81% | Val Loss: 0.0193, Val Acc: 91.80% | Time: 34.48s
Epoch [05/15] -> Train Loss: 0.0184, Train Acc: 92.35% | Val Loss: 0.0185, Val Acc: 92.23% | Time: 37.26s
Epoch [06/15] -> Train Loss: 0.0178, Train Acc: 92.80% | Val Loss: 0.0180, Val Acc: 92.70% | Time: 36.13s
Epoch [07/15] -> Train Loss: 0.0173, Train Acc: 92.98% | Val Loss: 0.0175, Val Acc: 92.88% | Time: 34.69s
Epoch [08/15] -> Train Loss: 0.0169, Train Acc: 93.20% | Val Loss: 0.0174, Val Acc: 92.90% | Time: 35.81s
Epoch

# Plots

In [ ]:
def plot_comparison(bp_history, pc_history):
    """Generates the 2x3 comparison plot matrix."""
    fig, axes = plt.subplots(2, 3, figsize=(20, 10))
    epochs = range(1, NUM_EPOCHS + 1)

    # --- Row 1: Backpropagation Results ---
    axes[0, 0].plot(epochs, bp_history["train_losses"], "b-o", label="Train Loss")
    axes[0, 0].plot(epochs, bp_history["val_losses"], "r-s", label="Val Loss")
    axes[0, 0].set_title("Backpropagation: Loss vs. Epoch")
    axes[0, 0].set_xlabel("Epoch")
    axes[0, 0].set_ylabel("Loss")
    axes[0, 0].legend()
    axes[0, 0].grid(False)

    axes[1, 0].plot(
        epochs, bp_history["train_accuracies"], "b-o", label="Train Accuracy"
    )
    axes[1, 0].plot(epochs, bp_history["val_accuracies"], "r-s", label="Val Accuracy")
    axes[1, 0].set_title("Backpropagation: Accuracy vs. Epoch")
    axes[1, 0].set_xlabel("Epoch")
    axes[1, 0].set_ylabel("Accuracy (%)")
    axes[1, 0].legend()
    axes[1, 0].grid(False)

    # --- Row 2: Predictive Coding Results ---
    axes[0, 1].plot(epochs, pc_history["train_losses"], "b-o", label="Train Loss")
    axes[0, 1].plot(epochs, pc_history["val_losses"], "r-s", label="Val Loss")
    axes[0, 1].set_title("Predictive Coding: Loss vs. Epoch")
    axes[0, 1].set_xlabel("Epoch")
    axes[0, 1].set_ylabel("Loss")
    axes[0, 1].legend()
    axes[0, 1].grid(False)

    axes[1, 1].plot(
        epochs, pc_history["train_accuracies"], "b-o", label="Train Accuracy"
    )
    axes[1, 1].plot(epochs, pc_history["val_accuracies"], "r-s", label="Val Accuracy")
    axes[1, 1].set_title("Predictive Coding: Accuracy vs. Epoch")
    axes[1, 1].set_xlabel("Epoch")
    axes[1, 1].set_ylabel("Accuracy (%)")
    axes[1, 1].legend()
    axes[1, 1].grid(False)

    # --- Row 3: Combined Results ---
    axes[0, 2].plot(epochs, bp_history["train_losses"], "b-o", label="BP Train Loss")
    axes[0, 2].plot(epochs, bp_history["val_losses"], "r-s", label="BP Val Loss")
    axes[0, 2].plot(epochs, pc_history["train_losses"], "b--o", label="PC Train Loss")
    axes[0, 2].plot(epochs, pc_history["val_losses"], "r--s", label="PC Val Loss")
    axes[0, 2].set_title("Combined: Loss vs. Epoch")
    axes[0, 2].set_xlabel("Epoch")
    axes[0, 2].set_ylabel("Loss")
    axes[0, 2].legend()
    axes[0, 2].grid(False)

    axes[1, 2].plot(epochs, bp_history["train_accuracies"], "b-o", label="BP Train Acc")
    axes[1, 2].plot(epochs, bp_history["val_accuracies"], "r-s", label="BP Val Acc")
    axes[1, 2].plot(
        epochs, pc_history["train_accuracies"], "b--o", label="PC Train Acc"
    )
    axes[1, 2].plot(epochs, pc_history["val_accuracies"], "r--s", label="PC Val Acc")
    axes[1, 2].set_title("Combined: Accuracy vs. Epoch")
    axes[1, 2].set_xlabel("Epoch")
    axes[1, 2].set_ylabel("Accuracy (%)")
    axes[1, 2].legend()
    axes[1, 2].grid(False)

    plt.tight_layout()
    plt.savefig("comparison_plot_2.png", dpi=300)
    plt.show()

In [ ]:
plot_comparison(bp_history, pc_history)

In [ ]:
pd.DataFrame(pc_history)